# CS260R Reinforcement Learning Assignment 0: Jupyter Notebook usage and assignment submission workflow

------

*CS260R: Reinforcement Learning. Department of Computer Science at University of California, Los Angeles.
Course Instructor: Professor Bolei ZHOU. Assignment author: Zhenghao PENG.*


You are asked to finish four tasks:

1. Fill in your name and University ID in the next cell.
2. Install pytorch and finish the [Kindergarten Pytorch](Kindergarten Pytorch) section.
3. Run all cells and export this notebook **as a PDF file**.
4. Compress this folder `assignment0` **as a ZIP file** and submit **the PDF file and the ZIP file separately as two files** in BruinLearn.

In [2]:
# TODO: Fill your name and UID here
my_name = "lucas"
my_student_id = "1"

In [3]:
# Run this cell without modification

text = "Oh, I finished this assignment! I am {} ({})".format(my_name, my_student_id)
print(text)
with open("{}.txt".format(text), "w") as f:
    f.write(text)

Oh, I finished this assignment! I am lucas (1)


## Kindergarten Pytorch

1. Please install pytorch in your virtual environment following the instruction: [https://pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/).
```
pip install torch torchvision
```
2. If you are not familiar with Pytorch, please go through [the tutorial in official website](https://pytorch.org/tutorials/beginner/basics/intro.html) until you can understand the [quick start tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).
3. The following code is copied from the [quick start tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html), please solve all `TODO`s and print the result in the cells before generating the PDF file.


In [7]:
# You can also run this cell in notebook:

!pip install torch torchvision

  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.1 MB 2.8 MB/s eta 0:01:13
   ---------------------------------------- 1.3/204.1 MB 4.2 MB/s eta 0:00:49
    --------------------------------------- 3.9/204.1 MB 6.9 MB/s eta 0:00:29
   - -------------------------------------- 8.9/204.1 MB 11.5 MB/s eta 0:00:17
   - -------------------------------------- 9.4/204.1 MB 11.7 MB/s eta 0:00:17
   -- ------------------------------------- 12.3/204.1 MB 10.4 MB/s eta 0:00:19
   --- ------------------------------------ 18.1/204.1 MB 13.8 MB

### Prepare data

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

### Define model

In [10]:
torch.cuda.is_available()

False

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        # TODO: Define the self.linear_relu_stack by uncommenting next few lines
        # and understand what they mean
        self.linear_relu_stack = nn.Sequential(
             nn.Linear(28 * 28, 512),
             nn.ReLU(),
             nn.Linear(512, 512),
             nn.ReLU(),
             nn.Linear(512, 10)
         )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Define training and test pipelines

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation

        # TODO: Uncomment next three lines and understand what they mean
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # TODO: Uncomment next line and understand what it means
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Run the training and test pipelines

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310121  [   64/60000]
loss: 2.301378  [ 6464/60000]
loss: 2.277304  [12864/60000]
loss: 2.267275  [19264/60000]
loss: 2.255785  [25664/60000]
loss: 2.224964  [32064/60000]
loss: 2.233045  [38464/60000]
loss: 2.198854  [44864/60000]
loss: 2.197069  [51264/60000]
loss: 2.168908  [57664/60000]
Test Error: 
 Accuracy: 46.9%, Avg loss: 2.154956 

Epoch 2
-------------------------------
loss: 2.158357  [   64/60000]
loss: 2.155909  [ 6464/60000]
loss: 2.091932  [12864/60000]
loss: 2.112060  [19264/60000]
loss: 2.065409  [25664/60000]
loss: 2.005161  [32064/60000]
loss: 2.035177  [38464/60000]
loss: 1.957583  [44864/60000]
loss: 1.965641  [51264/60000]
loss: 1.902644  [57664/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.885901 

Epoch 3
-------------------------------
loss: 1.910269  [   64/60000]
loss: 1.887017  [ 6464/60000]
loss: 1.765958  [12864/60000]
loss: 1.810463  [19264/60000]
loss: 1.711876  [25664/60000]
loss: 1.660081  [32064/600

### Save model

In [8]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Load model and run the inference

In [9]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
